In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm



D:\Anaconda3\envs\vits\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#test torch
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.10.1+cu102
True


In [65]:
umaDict = {'特别周': 0, '无声铃鹿': 1, '东海帝皇': 2, '丸善斯基': 3, '富士奇迹': 4,
           '小栗帽': 5, '黄金船': 6, '伏特加': 7, '大和赤骥': 8, '大树快车': 9, '草上飞': 10,
           '菱亚马逊': 11, '目白麦昆': 12, '神鹰': 13, '好歌剧': 14, '成田白仁': 15, '鲁道夫象征（皇帝）': 16, 
           '气槽': 17, '爱丽数码': 18, '星云天空': 19, '玉藻十字': 20, '美妙姿势': 21, '琵琶晨光': 22, '摩耶重炮': 23, 
           '曼城茶座': 24, '美浦波旁': 25, '目白赖恩': 26, '菱曙': 27, '雪中美人': 28, '米浴': 29, '艾尼斯风神': 30,
           '爱丽速子': 31, '爱慕织姬': 32, '稻荷一': 33, '胜利奖券': 34, '空中神宫': 35, '荣进闪耀': 36, '真机伶': 37,
           '川上公主': 38, '黄金城': 39, '樱花进王': 40, '采珠': 41, '新光风': 42, '东商变革': 43, '超级小海湾': 44,
           '醒目飞鹰': 45, '荒漠英雄': 46, '东瀛佐敦': 47, '中山庆典': 48, '成田大进': 49, '西野花': 50, '春乌拉拉': 51, 
           '青竹回忆': 52, '微光飞驹': 53, '美丽周日': 54, '待兼福来': 55, 'Mr CB': 56, '名将怒涛': 57, '目白多伯': 58,
           '优秀素质': 59, '帝王光辉': 60, '待兼诗歌剧': 61, '生野狄杜斯': 62, '目白善信': 63, '大拓太阳神': 64, 
           '双涡轮': 65, '里见光钻': 66, '北部玄驹': 67, '樱花千代王': 68, '天狼星象征': 69, '目白阿尔丹': 70,
           '八重无敌': 71, '鹤丸刚志': 72, '目白光明': 73, '成田路': 74, '也文摄辉': 75, '小林力奇': 76,
           '北港火山': 77, '奇瑞骏': 78, '苦涩糖霜': 79, '小小蚕茧': 80, '骏川手纲': 81, '秋川弥生': 82,
           '乙名史悦子': 83, '桐生院葵': 84, '安心泽刺刺美': 85, '樫本理子': 86}
configPath = "./logs/uma.json"
pthPath = "./logs/uma466.pth"
outputPath = "./output/output.wav"
#读取角色
#角色名
ID=umaDict['里见光钻']


hps_ms = utils.get_hparams_from_file(configPath)
net_g = SynthesizerTrn(
    len(hps_ms.symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
_ = net_g.eval()

_ = utils.load_checkpoint(pthPath, net_g, None)

outputWav = open(outputPath,'w')

INFO:root:Loaded checkpoint './logs/uma466.pth' (iteration 212)


In [66]:
#TODO:实现 说话人|句子的结构 一行一行地读。
words__ = open("words.txt",'r',encoding="UTF-8")
words_ = words__.readlines()
words = words_.copy()
for i in range(len(words)):
    if words_[i] == "\n" or words_[i] =="":
        continue
    words[i] = words_[i].split('|')
print(words)

[['里见光钻', '「歩夢が部屋に来るの\u3000久しぶりだね」\n'], '\n', ['北部玄驹', '「そうだね」\n'], '\n', ['北部玄驹', '「あっ…！」\n'], '\n', ['里见光钻', '「ああ\u3000これ？\u3000少し前から練習してるんだけど\u3000全然うまくならなくって」\n'], '\n', ['里见光钻', '「あのね\u3000歩夢に話そうと思ってたことがあるんだ」\n'], '\n', ['里见光钻', '「ただ\u3000自分でも自信が持てなくて\u3000もっと弾けるようになってからって思ってたら\u3000時間たっちゃってさ」\n'], '\n', ['北部玄驹', '「それってピアノのこと？」\n'], '\n', ['里见光钻', '「え？\u3000うん\u3000それもあるんだけど…」\n'], '\n', ['北部玄驹', '「だったら\u3000どうしてせつ菜ちゃんには教えたの？」\n'], '\n', ['北部玄驹', '「私には言えなくて\u3000せつ菜ちゃんには…！」\n'], '\n', ['里见光钻', '「え？\u3000なんでせつ菜ちゃんが出てくるの？」\n'], '\n', ['北部玄驹', '「せつ菜ちゃんの方が大事なの！？」\n'], '\n', ['里见光钻', '「違うよ」\n'], '\n', ['北部玄驹', '「あっ…！」\n'], '\n', ['里见光钻', '「歩夢に伝えたかったのは…」\n'], '\n', ['北部玄驹', '「ん？」\n'], '\n', ['里见光钻', '「もっと\u3000先のこと」\n'], '\n', ['北部玄驹', '「え…？」\n'], '\n', ['里见光钻', '「私ね\u3000夢ができ…」\n'], '\n', ['北部玄驹', '「嫌！」\n'], '\n', ['里见光钻', '「歩夢！？」\n'], '\n', ['北部玄驹', '「聞きたくないよ…」\n'], '\n', ['里见光钻', '「歩夢？」\n'], '\n', ['北部玄驹', '「私の夢を一緒に見てくれるって\u3000ずっと隣にいてくれるって\u3000言ったじゃない…！」\n'], '\n', ['里见光钻', '「あ…」\n

In [59]:

words_ = open("words.txt",'r',encoding="UTF-8")
words = words_.read()
print(words)
text=words
#CPU推理
stn_tst = get_text(text, hps_ms)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid = torch.LongTensor([ID])
    audio_ = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
audio = ipd.Audio(audio_, rate=hps_ms.data.sampling_rate, normalize=False)
ipd.display(audio)
print(audio)
write("./output/output.wav",hps_ms.data.sampling_rate,audio_)

きたちゃん！


<IPython.lib.display.Audio object>


In [15]:
#GPU推理
words_ = open("words.txt",'r',encoding="UTF-8")
words = words_.read()
print(words)
text=words



hps_ms = utils.get_hparams_from_file(configPath)
net_g = SynthesizerTrn(
    len(hps_ms.symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint(pthPath, net_g, None)

stn_tst = get_text(text, hps_ms)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([ID]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate, normalize=False))

ここのつここのかここのいろ
INFO:root:Loaded checkpoint './logs/uma412.pth' (iteration 186)
